# DnCNN

In [1]:
import os
import random
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.metrics import mean_squared_error as mse

- ノイズ付加関数

In [2]:
def add_noise(raw):
    noise = np.array([(-2*v)*np.random.rand()+v for v in raw.flatten()]).reshape(raw.shape[0],raw.shape[1])
    noised_raw = noise + raw
    return noised_raw

- データ読み込み

In [3]:
val = 80
def load_batch(n_batches=1,is_testing=False):

    base_path = '../input/raw/'
    
    if not is_testing:
        path = sorted(os.listdir(base_path))[:val]
    else:
        path = sorted(os.listdir(base_path))[val:]
    
    # x_data:ノイズありmel, y_data:ノイズなしmel
    x_data, y_data = [], []
    for _ in range(n_batches):
        for mel_path in path:
            raw = np.load(base_path + mel_path)
            
            for i in range(0,raw.shape[0]-mel_shape[0],3):
                for j in range(0,raw.shape[1]-mel_shape[1],3):
                    y = raw[i:i+mel_shape[0],j:j+mel_shape[1]]
                    x = add_noise(y)
                    
                    x = x.reshape(mel_shape[0],mel_shape[1],1)
                    y = y.reshape(mel_shape[0],mel_shape[1],1)
                    
                    x_data.append(x)
                    y_data.append(y)

    return  np.array(x_data), np.array(y_data)

- モデル読み込み

In [4]:
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Lambda,Subtract

def create_dncnn():
    inpt = Input(shape=(mel_shape[0],mel_shape[1],1))
    # 1st layer, Conv+relu
    x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(inpt)
    x = Activation('relu')(x)
    # 15 layers, Conv+BN+relu
    for i in range(15):
        x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
        x = BatchNormalization(axis=-1, epsilon=1e-3)(x)
        x = Activation('relu')(x)   
    # last layer, Conv
    x = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = Subtract()([inpt, x])   # input - noise
    model = Model(inputs=inpt, outputs=x)
    
    return model

In [5]:
mel_shape = (32,32) # input_shape

In [6]:
x_train, y_train = load_batch(n_batches=1,is_testing=False)
x_test, y_test = load_batch(n_batches=1,is_testing=True)

In [7]:
model = create_dncnn()
model.compile(optimizer='adam',loss='mse')

In [8]:
model.fit(x_train,y_train,epochs=3,validation_data=(x_test,y_test))

Train on 153472 samples, validate on 41184 samples
Epoch 1/3
153472/153472 [==============================] - 124s 809us/sample - loss: 0.9236 - val_loss: 0.8657
Epoch 2/3
153472/153472 [==============================] - 119s 778us/sample - loss: 0.6407 - val_loss: 0.6177
Epoch 3/3
153472/153472 [==============================] - 119s 777us/sample - loss: 0.5639 - val_loss: 0.6149


- noisedデータにモデルを適用

In [16]:
def fitting(noised_tgt):
    vector_dict = {}
    for i in range(noised_tgt.shape[0]):
        for j in range(noised_tgt.shape[1]):
            vector_dict[(i,j)] = []

    x_test = []
    for i in range(0,noised_tgt.shape[0]-mel_shape[0],5):
        for j in range(0,noised_tgt.shape[1]-mel_shape[1],5):
            v = noised_tgt[i:i+mel_shape[0],j:j+mel_shape[1]].reshape(mel_shape[0],mel_shape[1],1)
            x_test.append(v)

    x_test = np.array(x_test)
    v_noise = model.predict(x_test).reshape(-1,mel_shape[0],mel_shape[1])
    c = 0
    for i in range(0,noised_tgt.shape[0]-mel_shape[0],5):
        for j in range(0,noised_tgt.shape[1]-mel_shape[1],5):
            for i2 in range(v_noise[0].shape[0]):
                for j2 in range(v_noise[0].shape[1]):
                    vector_dict[(i+i2,j+j2)].append(v_noise[c][i2][j2])
            c += 1
            
    pred = np.copy(noised_tgt)
    for i in range(noised_tgt.shape[0]):
        for j in range(noised_tgt.shape[1]):
            if len(vector_dict[(i,j)]) > 0:
                value = np.mean(vector_dict[(i,j)])
                if value > 0:
                    pred[i][j] = value

    return pred

- val_dataに対して実行

In [17]:
base_path = '../input/raw/'
path = sorted(os.listdir(base_path))[val:]

targets = []
noises = []
preds = []

for mel_path in path:
    raw = np.load(base_path + mel_path)
    noised_raw = add_noise(raw)
    pred = fitting(noised_raw)
    
    targets.append(raw)
    noises.append(noised_raw)
    preds.append(pred)
    
    
print('before:',np.mean([mse(targets[i].flatten(),noises[i].flatten()) for i in range(len(targets))]))
print('after :',np.mean([mse(targets[i].flatten(),preds[i].flatten()) for i in range(len(targets))]))

before: 4.316797298967814
after : 0.7051285473010405


- submit_dataに対して実行

In [18]:
name = '002_dncnn'
path = '../output/unzip_data/%s/' % (name)

os.makedirs(path, exist_ok=True)
noised_path = '../input/noised_tgt/'

for p in sorted(os.listdir(noised_path)):
    noised_tgt = np.load(noised_path+p)
    clean_tgt = fitting(noised_tgt)
                    
    np.save(path+p[7:],clean_tgt)

- dataをzip化

In [19]:
import shutil
base = '../output/'
unzip_path = base+'unzip_data/'+name
zip_path = base+'zip_data/'+name
shutil.make_archive(zip_path,"zip",root_dir = unzip_path)

'/home/share/BandaiNamco-DSChallenge/output/zip_data/002_dncnn.zip'